In [1]:
import gensim.downloader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
from gensim.models import KeyedVectors
from keras.layers import Dense, Flatten, LSTM, GRU, Bidirectional, Embedding
from keras.models import Sequential
from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, classification_report
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data\preprocessed.csv').dropna()
data = data[((data[" 'American'"]==1) | (data["'American'"]==1))]
nlp = spacy.load("en_core_web_sm")
#glove = gensim.downloader.load('glove-twitter-200')
#glove.save("glove.model")
glove = KeyedVectors.load("glove.model")

In [3]:
words = list(glove.key_to_index.keys())
embeddings_index = {}
for word in words:    
    coefs = np.asarray(glove[word], dtype='float32')
    embeddings_index[word] = coefs
print(len(embeddings_index))

1193514


In [4]:
X = data["Reviews_cleaned"]
y = data["Category"]

In [5]:
maxlen = 10
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
print(f"Znaleziono {len(word_index)} unikatowych tokenów.")
data_X = sequence.pad_sequences(sequences, maxlen=maxlen)
print(f"Kształt tensora danych: {data_X.shape}") 
print(f"Kształt tensora etykiet: {y.shape}")

Znaleziono 1309 unikatowych tokenów.
Kształt tensora danych: (1826, 10)
Kształt tensora etykiet: (1826,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data_X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [8]:
X_train.shape, X_test.shape, X.shape

((1460, 10), (366, 10), (1826,))

In [9]:
embedding_dim = 200
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            embedding_matrix[i] = glove['unk']

In [10]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 200)           2000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 128)           135680    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9

In [11]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(X_train, y_train,
                    epochs=5,
                    batch_size=32,
                    validation_split=0.2)
#model.save_weights('pre_trained_glove_model.h5')

Epoch 1/5
37/37 [==============================] - 13s 71ms/step - loss: 0.6485 - acc: 0.5942 - val_loss: 0.5657 - val_acc: 0.7534
Epoch 2/5
37/37 [==============================] - 2s 52ms/step - loss: 0.5971 - acc: 0.7414 - val_loss: 0.5625 - val_acc: 0.7055
Epoch 3/5
37/37 [==============================] - 2s 52ms/step - loss: 0.5637 - acc: 0.7705 - val_loss: 0.5124 - val_acc: 0.7432
Epoch 4/5
37/37 [==============================] - 2s 62ms/step - loss: 0.4713 - acc: 0.7920 - val_loss: 0.4844 - val_acc: 0.7637
Epoch 5/5
37/37 [==============================] - 2s 51ms/step - loss: 0.4275 - acc: 0.8219 - val_loss: 0.4867 - val_acc: 0.7705


In [13]:
y_pred = np.round(model.predict(X_test))
f1_score(y_test, y_pred)

0.7788461538461539

In [14]:
cohen_kappa_score(y_test, y_pred)

0.48994183228308297